# Observed Air Quality (PurpleAir)

This notebook retrieves readings from PurpleAir Sensors in Minneapolis and cleans the entries and saves the results as a csv file.

Documentation is available here: https://api.purpleair.com.
You can read this article for help getting started: https://community.purpleair.com/t/making-api-calls-with-the-purpleair-api/180.

From PurpleAir: 

"The data from individual sensors will update no less than every 30 seconds. As a courtesy, we ask that you limit the number of requests to no more than once every 1 to 10 minutes, assuming you are only using the API to obtain data from sensors. If retrieving data from multiple sensors at once, please send a single request rather than individual requests in succession.

The PurpleAir historical API is released as of July 18, 2022. For more information, view this post: https://community.purpleair.com/t/new-version-of-the-purpleair-api-on-july-18th/1251.

Please let us know if you have any questions or concerns, and have a great day!"

A paper on this process: https://doi.org/10.5194/amt-14-4617-2021 (Link for [Download](https://www.researchgate.net/publication/352663348_Development_and_application_of_a_United_States-wide_correction_for_PM25_data_collected_with_the_PurpleAir_sensor) )

Chat on which PM Estimate to use: https://community.purpleair.com/t/pm2-5-algorithms/3972/6

In [2]:
### Import Packages

# File manipulation

import os # For working with Operating System
import requests # Accessing the Web
import datetime as dt # Working with dates/times
import io # Input/Output Bytes objects

# Analysis

import numpy as np
import pandas as pd

## Set Working Environment

In [3]:
# # Get CWD

# cwd = os.getcwd() # This is a global variable for where the notebook is (must change if running in arcpro)

# # Create GeoDataBase
# # This is the communal GeoDataBase

# if not os.path.exists(os.path.join(cwd, '..', '..', 'data', 'QAQC.gdb')): # If it doesn't exist, create it

#     arcpy.management.CreateFileGDB(os.path.join(cwd, '..', '..', 'data'), 'QAQC')

# # Make it workspace

# arcpy.env.workspace = os.path.join(cwd, '..', '..', 'data', 'QAQC.gdb')

# arcpy.env.overwriteOutput = True # Overwrite layers is okay

## Definitions

In [4]:
spike_threshold = 15 # Micgrograms per meter cubed

### Summary Statistics Functions

In [23]:
%run Summary_Functions.py

print(summary_stats)
print(summary_stats_dtypes)
print(summary_stats_functions)

['n_observations']
[<class 'int'>]
[<function n_observations at 0x7f7c88dcf370>]


### Functions

In [24]:
# QAQC

def qaqc(df):
    '''This function will perform some basic QAQC
    '''
    
    clean_df = df.copy()
    
    # Convert timestamp to datetime
    
    clean_df['timestamp'] = pd.to_datetime(clean_df['timestamp'], unit='s')
    
    # Remove NaNs
    
    clean_df = clean_df.dropna()
    
    return clean_df

# Remove and record Spikes

def get_spikes(df, spike_threshold):
    '''This function removes spikes from a dataframe 
    and returns both the new dataframe
    and a separate spike dataframe
    '''
    
    quantile_95 = np.quantile(df.pm25.values, 0.95)
    
    condition = (df.pm25 > spike_threshold)
    
    spikes = df[condition]
    
    # What if the day had particularly high readings?
    
    if spike_threshold < quantile_95: # If true, we should include in analysis
    
        condition = (df.pm25 > quantile_95) # Update the condition to use quantile
    
    not_spikes = df[~condition]        
    
    return not_spikes, spikes

# Get Summary Stats

def get_summary_stats(df):
    ''' This is the main function. It will run all of our functions that get summary stats
    and return as a list.
    '''
    
    stats = []
    
    # Run the functions
    
    for f in summary_stats_functions:
        stats += f(df)
    
    return stats

### Set Up Parameters for Query

In [11]:
# This is my personal API key... Please use responsibly!

api = input('Please enter your Purple Air api key')

Please enter your Purple Air api key 51592903-B445-11ED-B6F4-42010A800007


In [12]:
### Query Strings

# Average string (in minutes) 1440 is 1 day average

avg_string = 'average=10'

# Environmental fields

env_fields = ['pm2.5_cf_1']

env_fields_string = 'fields=' + '%2C%20'.join(env_fields)

# My Header

my_headers = {'X-API-Key': api}

## The Loop

In [13]:
## Iterables

sensor_ids = [3088, 5582, 11134, 142718, 142720] # This should be an iterable of the sensor ids as integers

datelist = pd.date_range(start = dt.datetime(2022,6,15), # June 15, 2022,
                         end = dt.datetime.today(),
                        normalize = True)

print('Last Run on ', dt.datetime.today())

Last Run on  2023-04-19 18:37:00.407138


In [14]:
## Initialize Storage

# Daily Summary

cols = ['date'] + ['sensor_index'] + summary_stats

datatypes = [str, int] + summary_stats_dtypes

dtypes = np.dtype(list(zip(cols, datatypes)))

daily_summary_df = pd.DataFrame(np.empty(0, dtype = dtypes))

# Spikes

all_spikes_df = pd.DataFrame(np.empty(0, dtype = [('timestamp', pd._libs.tslibs.timestamps.Timestamp),
                                              ('pm25', float),
                                              ('sensor_index', int)]
                                 )
                        )

In [15]:
# Iterate through the days

for i in range(len(datelist)-1):
    
    # Set up Timestamp for query    
    
    start_timestamp = int(datelist[i].timestamp())
    end_timestamp = int(datelist[i+1].timestamp())
    
    time_string = 'start_timestamp=' + str(start_timestamp) + '&end_timestamp=' + str(end_timestamp)
    
    # Iterate through the Sensors
    
    for sensor_id in sensor_ids:
        # Base URL
        base_url = f'https://api.purpleair.com/v1/sensors/{sensor_id}/history/csv?'

        # Put it all together
        query_url = base_url + '&'.join([time_string, avg_string, env_fields_string])

        response = requests.get(query_url, headers=my_headers)

        if response.status_code == 200:

            # Read response as CSV data
            csv_data = response.content.decode('utf-8')

            if csv_data.count('\n') == 1: # There is only one line (empty data)
                print(f"No data for sensor {sensor_id} on {datelist[i]}")
                
            else:
                # Parse CSV data into pandas DataFrame
                df_individual_sensor = pd.read_csv(io.StringIO(csv_data),
                                                   header=0
                                                  )[['time_stamp', 'pm2.5_cf_1']]
                
                df_individual_sensor.columns = ['timestamp', 'pm25']
                
                # Perform QAQC
                
                clean = qaqc(df_individual_sensor)
                
                # Remove Spikes & Concatenate to main storage of spikes

                clean_no_spikes, spikes = get_spikes(clean, spike_threshold)
                
                spikes['sensor_index'] = int(sensor_id)
                
                all_spikes_df = pd.concat([all_spikes_df, spikes], ignore_index=True)
                
                # Get Stats

                sum_stats = get_summary_stats(clean_no_spikes)
                
                # Add to the daily summary dataframe
                
                row = [str(datelist[i].date()), int(sensor_id)] + sum_stats
                
                daily_summary_df.loc[len(daily_summary_df.index)] = row
        else:
            print(f"Error fetching data for sensor {sensor_id}: {r.status_code} on {datelist[i]}")
            
            
        break
    break

TypeError: 'int' object is not iterable

In [ ]:
daily_summary_df

In [31]:
all_spikes_df

,timestamp,pm25,sensor_index


In [23]:
# # Save a test dataframe

# clean_no_spikes.to_csv('example_df.csv', index = False)

# Save it!!!

# daily_summary_df.to_csv('daily_summaries.csv')